<a href="https://colab.research.google.com/github/etarazonav/650044-ABD-ULIMA/blob/main/Notebooks/ABD_RDD_Transformaciones_acciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" /> Transformaciones y Acciones usando PySpark
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>


Si se utiliza un entorno en el que no está instalado por defecto Spark, es neceario instalarlo (usando pip) y luego crear un contexto de Spark. En este caso, el contexto de Spark se crea con un solo hilo en local, y el nombre de la aplicación es `ejemplo_transformaciones`

In [ ]:
!pip install -q pyspark

In [ ]:
from pyspark import SparkContext

sc = SparkContext("local[1]", "ejemplo_transformaciones")

## 1.&nbsp;Transformaciones

### 1.1. Map

Se utiliza para realizar un mapeo de un RDD a otro RDD término a término. El mapeo se especifica mediante una función, que puede ser anónima (`lambda`) o puede haber sido definida usando `def`.

In [ ]:
# Creación de un RDD
rdd = sc.parallelize([1, 4, 6, 3])

# Map: se sumará 10 a cada elemento del RDD
rdd2 = rdd.map(lambda x: x+10)

# Mostrar el contenido del RDD creado usando map
rdd2.collect()

El siguiente ejemplo crea un RDD conteniendo un conjunto de cadenas de caracteres. Luego, se utiliza un `map` para obtener la cantidad de letras que tiene cada cadena de caracteres.

In [ ]:
# Creación del RDD
rdd = sc.parallelize(['este es un texto', 'este es otro texto de prueba', 'prueba 1'])

# Map: obtiene las cantidades de letras de cada cadena de caracteres (usando "len")
rdd2 = rdd.map(lambda x: len(x))

# Se muestra el resultado (utilizando la acción "collect")
rdd2.collect()

In [ ]:
# De manera alternativa se puede realizar el map y mostrarlo en una sola instrucción
rdd.map(lambda x: len(x)).collect()

### 1.2. Filter (filtro)

* Ejemplo 1: Filtro que solo mantiene aquellos valores mayores o iguales a 6. En este caso se utiliza una función anónima `lambda`.

In [ ]:
# Creación de un RDD
rdd = sc.parallelize([1, 4, 6, 3, 10, 15, 16])

# Filtro: solo mantiene aquellos elementos del RDD mayores o iguales a 6
rdd2 = rdd.filter(lambda x: x>=6)

# Mostrar el resultado usando la acción "collect"
rdd2.collect()

* Ejemplo 2: Filtro que solo mantiene aquellas cadenas de caracteres que tengan menos de 20 caracteres

In [ ]:
# Creación del RDD
rdd = sc.parallelize(['este es un texto', 'este es otro texto de prueba', 'prueba 1'])

# Función que retorna el valor de entrada si tiene una longitud menor a 20
def fmenor20(x):
    if (len (x) < 20):
        return x

# Filtro aplicando la función anterior
rdd.filter(fmenor20).collect()

### 1.3. Map y FlatMap

In [ ]:
# Creación de un RDD
rdd = sc.parallelize([1, 4, 6, 3])

* Ejemplo 1: Aplicación de un `map` o `flatMap` para multiplicar a cada elemento por 10 creando como salida una lista formada por el valor original y el valor multiplicado por 10

In [ ]:
# Aplicación de un mapa
rdd_map = rdd.map(lambda x: [x, 10*x])

# La salida mantiene la estructura deseada de la lista: [x, 10x]
rdd_map.collect()

In [ ]:
# Aplicación de un flatmap
rdd_flatmap = rdd.flatMap(lambda x: [x, 10*x])

# La salida no mantiene la estructura de lista
rdd_flatmap.collect()

* Ejemplo 2: separar una cadena de caracteres en palabras.

In [ ]:
# Creación del rdd
rdd = sc.parallelize(["hola mundo", "hoy es jueves"])

rdd.collect()

In [ ]:
# Aplicación de map
rdd_map = rdd.map(lambda x: x.split(" "))
rdd_map.collect()

In [ ]:
# Aplicación de flatmap
rdd_flatmap = rdd.flatMap(lambda x: x.split(" "))
rdd_flatmap.collect()

### 1.4. GroupByKey

In [ ]:
rdd = sc.parallelize([('a',7), ('b',4), ('a',1), ('b',6), ('c',3)])
rdd.collect()

In [ ]:
# Aplicación de agrupamiento por clave
rdd2 = rdd.groupByKey()

rdd2.collect()

Como se observa, al utilizar `groupByKey`, los valores se encuentran en formato de iterable de Spark. Para poder visualizarlos se debe convertir estos valores iterables en una lista. Esta conversión se puede realizar utilizando `mapValues(list)` que mapea solo los valores a listas

In [ ]:
rdd2.mapValues(list).collect()

In [ ]:
# El resultado es similar al caso anterior
rdd2.mapValues(lambda v: list(v)).collect()

Alternativamente, se puede utilizar el método `.data` de los iterables, que permiten recuperar los datos. Con este fin, se puede utilizar nuevamente `mapValues`.

In [ ]:
rdd2.mapValues(lambda v: v.data).collect()

### 1.5. Cogroup

In [ ]:
rdd = sc.parallelize([('a',7), ('b',4), ('a',1), ('b',6), ('c',3)])
rdd2 = sc.parallelize([('a',5), ('b',9)])

rdd3 = rdd.cogroup(rdd2)
rdd3.collect()

En este caso, los valores de cada elemento son tuplas de dos iterables. Por tanto, se puede utilizar `mapValues` para mapear a cada valor una conversión a lista, o el uso de `.data`, que también recupera los datos del iterable.

In [ ]:
rdd3.mapValues(lambda x: (x[0].data, x[1].data)).collect()

### 1.6. Join

In [ ]:
rdd = sc.parallelize([('a',7), ('b',4), ('a',1), ('b',6), ('c',3)])
rdd2 = sc.parallelize([('a',5), ('b',9), ('a',100)])

rdd3 = rdd.join(rdd2)
rdd3.collect()

### 1.7. Operadores de Conjuntos

In [ ]:
rdd1 = sc.parallelize([1, 3, 5, 7])
rdd2 = sc.parallelize([20, 7, 40, 60, 5])

rdd1.union(rdd2).collect()

In [ ]:
rdd1.intersection(rdd2).collect()

In [ ]:
rdd1.subtract(rdd2).collect()

In [ ]:
rdd1.cartesian(rdd2).collect()

### 1.8. ReduceByKey

Requiere que el RDD de entrada contenga tuplas, y actúa reduciendo los valores de las tuplas que tienen la misma clave.

In [ ]:
# RDD de entrada
rdd = sc.parallelize([('a',7), ('b',4), ('a',1), ('a',100), ('b',6), ('c',3)])
rdd.collect()

En este ejemplo, se suma los valores que tienen las mismas tuplas (como la reducción de MapReduce)

In [ ]:
rdd.reduceByKey(lambda x,y: x+y).collect()

### 1.9. SortByKey

Ordena según las claves. Requiere que el RDD utilizado tenga datos en forma de tuplas.

In [ ]:
rdd = sc.parallelize([('a',7), ('b',4), ('a',1), ('a',100), ('b',6), ('c',3)])
rdd.collect()

In [ ]:
rdd.sortByKey().collect()

In [ ]:
rdd.sortByKey(ascending=False).collect()

## 2.&nbsp;Acciones

### 2.1. Collect, take, top

In [ ]:
# Creación de un RDD
lista = [40, 20, 50, 10, 70, 30]
rdd = sc.parallelize(lista)

* `collect`: recupera todo el RDD y lo convierte a lista

In [ ]:
# Collect
rdd.collect()

* `take(n)`: toma `n` valores del RDD

In [ ]:
# Take
rdd.take(3)

* `top(n)`: Toma los `n` valores más altos del RDD

In [ ]:
# Top: 3 valores más altos
rdd.top(3)

### 2.2. Reduce

Reduce los valores según lo especificado por alguna función. En el siguiente ejemplo, reduce utilizando la suma de los elementos del RDD

In [ ]:
# Ejemplo usando la suma
rdd.reduce(lambda x,y: x+y)

### 2.3. saveAsTextFile

Almacena el RDD como un archivo de texto

In [ ]:
# Creación del RDD
rdd = sc.parallelize(["Este es un archivo", "de texto de ejemplo"])

# Almacenamiento del RDD (especificando el nombre de la carpeta)
rdd.saveAsTextFile("rdd_salida")

### 2.4. Ejemplo de procesamiento usando MapReduce

In [ ]:
# Creación de un RDD
rdd = sc.parallelize(['Hola este es un texto', 'este es un párrafo', 'es'])
rdd.collect()

In [ ]:
# Separación en palabras
rdd.flatMap(lambda x: x.split(" ")).collect()

In [ ]:
# Mapeo a 1 de cada palabra
rdd.flatMap(lambda x: x.split(" ")).map(lambda x: (x, 1)).collect()

In [ ]:
# Reducción: suma de valores para las mismas claves
rdd.flatMap(lambda x: x.split(" ")).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).collect()